In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd 

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Question 1

In [3]:
spark.version

'3.2.1'

Question 2

In [4]:
!curl -o fhvhv_tripdata_2021-02.csv https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  699M  100  699M    0     0  3387k      0  0:03:31  0:03:31 --:--:-- 2944k      0  0:03:09  0:00:20  0:02:49 3704k   0  0:03:16  0:01:19  0:01:57 3199k   0  3608k      0  0:03:18  0:01:30  0:01:48 3349k  3597k      0  0:03:19  0:01:34  0:01:45 3355k0  3579k      0  0:03:20  0:01:43  0:01:37 3387k  3504k      0  0:03:24  0:02:15  0:01:09 3321k502k      0  0:03:24  0:02:17  0:01:07 3321k  0  0:03:25  0:02:24  0:01:01 3317k     0  0:03:26  0:02:35  0:00:51 3304k 3444k      0  0:03:28  0:02:52  0:00:36 3369k0  3433k      0  0:03:28  0:02:58  0:00:30 3328k  0     0  3400k      0  0:03:30  0:03:21  0:00:09 3219k      0  0:03:30  0:03:27  0:00:03 3311k      0  0:03:31  0:03:31 --:--:-- 2885k


In [5]:
pd_df = pd.read_csv('fhvhv_tripdata_2021-02.csv', nrows = 100)

In [6]:
spark.createDataFrame(pd_df).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [7]:
from pyspark.sql import types

In [8]:
schema = types.StructType([
    types.StructField("hvfhs_license_num",types.StringType(),True),
    types.StructField("dispatching_base_num",types.StringType(),True),
    types.StructField("pickup_datetime",types.TimestampType(),True),
    types.StructField("dropoff_datetime",types.TimestampType(),True),
    types.StructField("PULocationID",types.IntegerType(),True),
    types.StructField("DOLocationID",types.IntegerType(),True),
    types.StructField("SR_Flag",types.StringType(),True)
])

In [9]:
df = spark.read\
    .option("header","true")\
    .schema(schema)\
    .csv('fhvhv_tripdata_2021-02.csv')

In [11]:
df.repartition(24).write.parquet('parquet/',mode='overwrite')

231.2 MB

question 3 


In [27]:
df = spark.read.parquet('parquet/')

In [28]:
from pyspark.sql import functions as F

In [29]:
df = df.withColumn('date', F.to_date(df.pickup_datetime))

In [43]:
df.createOrReplaceTempView('homework_data')

In [32]:
spark.sql("""
SELECT COUNT(*) FROM homework_data WHERE date = "2021-02-15";
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



question 4

In [22]:
from pyspark.sql import functions as F

In [42]:
df = df.withColumn('time_difference', df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long"))

In [49]:
spark.sql("""

SELECT 
    date
FROM
    homework_data
WHERE
    time_difference = (SELECT 
            MAX(time_difference)
        FROM
            homework_data)
LIMIT 1;
""").show()

+----------+
|      date|
+----------+
|2021-02-11|
+----------+



question 5

In [37]:
df.groupby('dispatching_base_num').count().sort('count', ascending=False).show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
|              B02869| 429720|
|              B02887| 322331|
|              B02871| 312364|
|              B02864| 311603|
|              B02866| 311089|
|              B02878| 305185|
|              B02682| 303255|
|              B02617| 274510|
|              B02883| 251617|
|              B02884| 244963|
|              B02882| 232173|
|              B02876| 215693|
|              B02879| 210137|
|              B02867| 200530|
|              B02877| 198938|
+--------------------+-------+
only showing top 20 rows



question 6

In [52]:
df_locations = df\
    .withColumn('concat', F.concat_ws('-',df.PULocationID, df.DOLocationID))

In [54]:
df_locations.groupby('concat').count().sort('count', ascending=False).show()

+-------+-----+
| concat|count|
+-------+-----+
|  76-76|45041|
|  26-26|37329|
|  39-39|28026|
|  61-61|25976|
|  14-14|17934|
|129-129|14688|
|    7-7|14688|
|  42-42|14481|
|  37-37|14424|
|  89-89|13976|
|216-216|13716|
|  35-35|12829|
|132-265|12542|
| 188-61|11814|
|  95-95|11548|
|  36-37|11491|
|  37-36|11487|
| 61-188|11462|
| 61-225|11342|
|188-188|11308|
+-------+-----+
only showing top 20 rows

